# 🤖 NeoScore - Credit Scoring Models

**Autor**: Luca Camus  
**Fecha**: Enero 2026  
**Objetivo**: Entrenar modelos de ML para predecir riesgo crediticio

**Modelos a implementar**:
1. Logistic Regression (baseline interpretable)
2. Random Forest (ensemble robusto)
3. XGBoost (estado del arte)
4. **Random Forest - Solo Comportamiento** (sin variables de balance)

## 1. Configuración

In [ ]:
# Instalar dependencias
!pip install google-cloud-bigquery pandas matplotlib seaborn scikit-learn xgboost imbalanced-learn --quiet

In [ ]:
# Imports
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, roc_curve, confusion_matrix, 
    classification_report, precision_recall_curve,
    f1_score, accuracy_score
)

# XGBoost
from xgboost import XGBClassifier

# Imbalanced-learn (si hay desbalanceo)
from imblearn.over_sampling import SMOTE

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Configuración completa')

## 2. Cargar Datos

In [ ]:
# Cliente BigQuery
PROJECT_ID = 'scoring-bancario'
client = bigquery.Client(project=PROJECT_ID)

# Cargar datos
query = """
SELECT *
FROM `scoring-bancario.analisis_bancario.customer_features`
"""

df = client.query(query).to_dataframe()
print(f'📊 Dataset cargado: {df.shape[0]:,} clientes x {df.shape[1]} features')

## 3. Preparación de Features (¡Evitando Leakage!)

⚠️ **IMPORTANTE**: NO usar `preliminary_credit_score` como feature

In [ ]:
# Features PERMITIDAS (datos crudos, sin leakage)
FEATURES = [
    'age',                      # Demografía
    'avg_balance',              # Balance
    'last_balance',
    'min_balance',
    'max_balance',
    'total_spend',              # Gasto
    'avg_spend',
    'max_spend',
    'min_spend',
    'std_spend',
    'total_transactions',       # Actividad
    'days_active',
    'unique_transaction_days',
    'transaction_frequency',
    'spend_to_balance_ratio',   # Ratios
    'spend_volatility',
    'avg_daily_transactions',
    'avg_daily_spend',
]

# Variable objetivo
TARGET = 'high_risk_flag'

print(f'📊 Features a usar: {len(FEATURES)}')
print(f'🎯 Variable objetivo: {TARGET}')

In [ ]:
# Verificar features disponibles
available_features = [f for f in FEATURES if f in df.columns]
missing_features = [f for f in FEATURES if f not in df.columns]

print(f'✅ Features disponibles: {len(available_features)}')
if missing_features:
    print(f'⚠️ Features no encontradas: {missing_features}')

FEATURES = available_features

In [ ]:
# Crear X e y
X = df[FEATURES].copy()
y = df[TARGET].copy()

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'\n📊 Distribución del target:')
print(y.value_counts(normalize=True).round(4) * 100)

In [ ]:
# Manejar valores nulos
print('\n📊 Nulos por columna antes de imputar:')
null_counts = X.isnull().sum()
print(null_counts[null_counts > 0])

# Imputar nulos con la mediana
X = X.fillna(X.median())

print('\n✅ Nulos después de imputar:', X.isnull().sum().sum())

## 4. División Train/Test

In [ ]:
# División estratificada (mantiene proporción de clases)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f'📊 Train: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.0f}%)')
print(f'📊 Test:  {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.0f}%)')

print(f'\n📊 Distribución en Train:')
print(y_train.value_counts(normalize=True).round(4) * 100)

print(f'\n📊 Distribución en Test:')
print(y_test.value_counts(normalize=True).round(4) * 100)

In [ ]:
# Escalar features para Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✅ Features escaladas')

## 5. Modelo 1: Logistic Regression (Baseline)

In [ ]:
# Entrenar Logistic Regression
lr_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

lr_model.fit(X_train_scaled, y_train)

# Predicciones
y_pred_lr = lr_model.predict(X_test_scaled)
y_prob_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Métricas
lr_auc = roc_auc_score(y_test, y_prob_lr)
lr_gini = 2 * lr_auc - 1

print('=' * 50)
print('📊 LOGISTIC REGRESSION - Resultados')
print('=' * 50)
print(f'ROC-AUC: {lr_auc:.4f}')
print(f'Gini:    {lr_gini:.4f}')
print(f'\n{classification_report(y_test, y_pred_lr, target_names=["Low Risk", "High Risk"])}')

In [ ]:
# Coeficientes (interpretabilidad)
coef_df = pd.DataFrame({
    'Feature': FEATURES,
    'Coeficiente': lr_model.coef_[0]
}).sort_values('Coeficiente', key=abs, ascending=False)

print('📊 Features más importantes (Logistic Regression):')
print(coef_df.head(10).to_string(index=False))

## 6. Modelo 2: Random Forest

In [ ]:
# Entrenar Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Métricas
rf_auc = roc_auc_score(y_test, y_prob_rf)
rf_gini = 2 * rf_auc - 1

print('=' * 50)
print('📊 RANDOM FOREST - Resultados')
print('=' * 50)
print(f'ROC-AUC: {rf_auc:.4f}')
print(f'Gini:    {rf_gini:.4f}')
print(f'\n{classification_report(y_test, y_pred_rf, target_names=["Low Risk", "High Risk"])}')

In [ ]:
# Feature Importance
importance_df = pd.DataFrame({
    'Feature': FEATURES,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualizar
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'][::-1], importance_df['Importance'][::-1], color='steelblue')
plt.xlabel('Importancia')
plt.title('Feature Importance - Random Forest', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\n📊 Top 10 Features (Random Forest):')
print(importance_df.head(10).to_string(index=False))

## 7. Modelo 3: XGBoost

In [ ]:
# Calcular scale_pos_weight para manejar desbalanceo
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f'Scale pos weight: {scale_pos_weight:.2f}')

# Entrenar XGBoost
xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='auc',
    use_label_encoder=False
)

xgb_model.fit(X_train, y_train)

# Predicciones
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Métricas
xgb_auc = roc_auc_score(y_test, y_prob_xgb)
xgb_gini = 2 * xgb_auc - 1

print('=' * 50)
print('📊 XGBOOST - Resultados')
print('=' * 50)
print(f'ROC-AUC: {xgb_auc:.4f}')
print(f'Gini:    {xgb_gini:.4f}')
print(f'\n{classification_report(y_test, y_pred_xgb, target_names=["Low Risk", "High Risk"])}')

## 8. 🎯 Modelo 4: Random Forest - SOLO COMPORTAMIENTO

⚠️ **PRUEBA DE ROBUSTEZ**: ¿El modelo puede predecir riesgo SIN conocer los balances?

**Variables EXCLUIDAS**:
- `avg_balance`, `last_balance`, `min_balance`, `max_balance`
- `spend_to_balance_ratio`

**Variables INCLUIDAS** (solo comportamiento):
- `age`, `avg_spend`, `total_spend`, `total_transactions`, `days_active`, etc.

In [ ]:
# Features SOLO DE COMPORTAMIENTO (sin balance)
FEATURES_BEHAVIOR = [
    'age',                      # Demografía
    'total_spend',              # Gasto
    'avg_spend',
    'max_spend',
    'min_spend',
    'std_spend',
    'total_transactions',       # Actividad
    'days_active',
    'unique_transaction_days',
    'transaction_frequency',
    'spend_volatility',
    'avg_daily_transactions',
    'avg_daily_spend',
]

# Variables EXCLUIDAS
EXCLUDED_FEATURES = [
    'avg_balance', 'last_balance', 'min_balance', 'max_balance',
    'spend_to_balance_ratio'
]

print('✅ Features de COMPORTAMIENTO (sin balance):')
for f in FEATURES_BEHAVIOR:
    print(f'   • {f}')

print(f'\n❌ Features EXCLUIDAS (tienen balance):')
for f in EXCLUDED_FEATURES:
    print(f'   ⚠️ {f}')

In [ ]:
# Filtrar features disponibles
FEATURES_BEHAVIOR = [f for f in FEATURES_BEHAVIOR if f in df.columns]

# Crear X para comportamiento
X_behavior = df[FEATURES_BEHAVIOR].copy()
X_behavior = X_behavior.fillna(X_behavior.median())

print(f'📊 Features de comportamiento: {len(FEATURES_BEHAVIOR)}')

In [ ]:
# División Train/Test para modelo de comportamiento
X_train_beh, X_test_beh, y_train_beh, y_test_beh = train_test_split(
    X_behavior, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f'📊 Train: {X_train_beh.shape[0]:,} samples')
print(f'📊 Test:  {X_test_beh.shape[0]:,} samples')

In [ ]:
# Entrenar Random Forest - Solo Comportamiento
rf_behavior = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

rf_behavior.fit(X_train_beh, y_train_beh)

# Predicciones
y_pred_beh = rf_behavior.predict(X_test_beh)
y_prob_beh = rf_behavior.predict_proba(X_test_beh)[:, 1]

# Métricas
beh_auc = roc_auc_score(y_test_beh, y_prob_beh)
beh_gini = 2 * beh_auc - 1

print('=' * 60)
print('📊 RANDOM FOREST - SOLO COMPORTAMIENTO (Sin Balance)')
print('=' * 60)
print(f'ROC-AUC: {beh_auc:.4f}')
print(f'Gini:    {beh_gini:.4f}')
print(f'\n{classification_report(y_test_beh, y_pred_beh, target_names=["Low Risk", "High Risk"])}')

In [ ]:
# Feature Importance - Comportamiento
importance_beh_df = pd.DataFrame({
    'Feature': FEATURES_BEHAVIOR,
    'Importance': rf_behavior.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualizar
plt.figure(figsize=(10, 6))
plt.barh(importance_beh_df['Feature'][::-1], importance_beh_df['Importance'][::-1], color='coral')
plt.xlabel('Importancia')
plt.title('Feature Importance - Solo Comportamiento (Sin Balance)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\n📊 Features más importantes (Solo Comportamiento):')
print(importance_beh_df.to_string(index=False))

## 9. Comparación: Con Balance vs Sin Balance

In [ ]:
# KS
def calculate_ks(y_true, y_prob):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return max(tpr - fpr)

ks_lr = calculate_ks(y_test, y_prob_lr)
ks_rf = calculate_ks(y_test, y_prob_rf)
ks_xgb = calculate_ks(y_test, y_prob_xgb)
ks_beh = calculate_ks(y_test_beh, y_prob_beh)

# Tabla comparativa
comparison = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'Random Forest', 'XGBoost', 'RF Solo Comportamiento'],
    'ROC-AUC': [lr_auc, rf_auc, xgb_auc, beh_auc],
    'Gini': [lr_gini, rf_gini, xgb_gini, beh_gini],
    'KS': [ks_lr, ks_rf, ks_xgb, ks_beh],
    'Usa Balance': ['Sí', 'Sí', 'Sí', 'NO']
}).round(4)

print('=' * 70)
print('📊 COMPARACIÓN: CON BALANCE vs SIN BALANCE')
print('=' * 70)
print(comparison.to_string(index=False))
print('=' * 70)

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = ['LR', 'RF', 'XGB', 'RF\n(Comportamiento)']
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
aucs = [lr_auc, rf_auc, xgb_auc, beh_auc]

# ROC-AUC
bars = axes[0].bar(models, aucs, color=colors)
axes[0].set_title('ROC-AUC por Modelo', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 1.1)
axes[0].axhline(0.7, color='gray', linestyle='--', alpha=0.5, label='Umbral aceptable (0.7)')
for i, v in enumerate(aucs):
    axes[0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')
axes[0].legend()

# Curvas ROC
fpr_beh, tpr_beh, _ = roc_curve(y_test_beh, y_prob_beh)

axes[1].plot(fpr_lr, tpr_lr, label=f'LR (AUC={lr_auc:.3f})', linewidth=2)
axes[1].plot(fpr_rf, tpr_rf, label=f'RF (AUC={rf_auc:.3f})', linewidth=2)
axes[1].plot(fpr_xgb, tpr_xgb, label=f'XGB (AUC={xgb_auc:.3f})', linewidth=2)
axes[1].plot(fpr_beh, tpr_beh, label=f'RF Comportamiento (AUC={beh_auc:.3f})', linewidth=2, linestyle='--')
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curvas ROC', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

## 10. Conclusiones

In [ ]:
print('=' * 70)
print('📊 CONCLUSIONES - NeoScore Credit Scoring')
print('=' * 70)

# Interpretación
drop_in_auc = rf_auc - beh_auc
pct_drop = (drop_in_auc / rf_auc) * 100

print(f'''
1. MODELOS CON BALANCE (RF Completo):
   - ROC-AUC: {rf_auc:.4f}
   - ⚠️ Puede estar "haciendo trampa" usando balance para predecir riesgo

2. MODELO SOLO COMPORTAMIENTO (RF Sin Balance):
   - ROC-AUC: {beh_auc:.4f}
   - Caída en AUC: {drop_in_auc:.4f} ({pct_drop:.1f}%)
   - Este es el RENDIMIENTO REAL del modelo sin "trampa"

3. INTERPRETACIÓN:
''')

if beh_auc >= 0.70:
    print('   ✅ El modelo de comportamiento tiene AUC >= 0.70')
    print('   ✅ PUEDE predecir riesgo sin conocer el balance')
    print('   ✅ El modelo es ROBUSTO y útil para producción')
elif beh_auc >= 0.60:
    print('   ⚠️ El modelo de comportamiento tiene AUC entre 0.60-0.70')
    print('   ⚠️ Capacidad predictiva MODERADA sin balance')
    print('   ⚠️ Considerar agregar más features de comportamiento')
else:
    print('   ❌ El modelo de comportamiento tiene AUC < 0.60')
    print('   ❌ El comportamiento SOLO no es suficiente para predecir riesgo')
    print('   ❌ El modelo original dependía demasiado del balance')

print(f'''
4. FEATURES MÁS IMPORTANTES (Sin Balance):
''')
print(importance_beh_df.head(5).to_string(index=False))

print('\n' + '=' * 70)